In [ ]:
#create the data for given problem

import pandas as pd
import numpy as np
import random
loan_size=[10,25,4,11,18,3,17,15,9,10]
interest=[0.021 ,0.022, 0.021, 0.027 ,0.025 ,0.026, 0.023, 0.021 ,0.028 ,0.022]
rating=['AAA', 'BB', 'A' ,'AA', 'BBB', 'AAA' ,'BB', 'AAA', 'A', 'A']
loss=[0.0002, 0.0058 ,0.0001, 0.0003, 0.0024, 0.0002, 0.0058 ,0.0002 ,0.001, 0.001]
customers_data=pd.DataFrame()
customers_data['loan size']=loan_size
customers_data['interest']=interest
customers_data['rating']=rating
customers_data['loss']=loss
customers_data


,loan size,interest,rating,loss
0,10,0.021,AAA,0.0002
1,25,0.022,BB,0.0058
2,4,0.021,A,0.0001
3,11,0.027,AA,0.0003
4,18,0.025,BBB,0.0024
5,3,0.026,AAA,0.0002
6,17,0.023,BB,0.0058
7,15,0.021,AAA,0.0002
8,9,0.028,A,0.0010
9,10,0.022,A,0.0010


In [ ]:
#check if the chromosome satisfies the constraint
def check_limit(selected_chromo_data,K,D):
  sum_loan_size=0
  for i in range(selected_chromo_data.shape[0]):
    sum_loan_size+=selected_chromo_data.iloc[i]['loan size']
     # print(sum_loan_size,"Threshold",(1-K)*D)
  return sum_loan_size<=((1-K)*D)


def generate_initial_population(population_size,customers_data,number_of_customers,K,D):
  initial_population=[]
  #create the initial population on chromosomes
  while len(initial_population)<population_size:
    chromosome=np.zeros(number_of_customers,dtype=bool)
    selected=[]
    for i in range(number_of_customers):
      rand=np.random.randint(1,100)
      if rand<=50:
        chromosome[i]=True
        selected.append(i)

    selected_chromo_data=customers_data.iloc[selected,:]

    if check_limit(selected_chromo_data,K,D):
      initial_population.append(chromosome)

  return np.array(initial_population)



In [ ]:
#revenue
def calc_revenue(selected_chromo_data):
  revenue=0
  for i in range(selected_chromo_data.shape[0]):
    L=selected_chromo_data.iloc[i]['loan size']
    rL=selected_chromo_data.iloc[i]['interest']
    lamda=selected_chromo_data.iloc[i]['loss']

    revenue+=(rL*L)-lamda
  return revenue

def calc_omega(selected_chromo_data,K,D,rT):
  omega=0
  for i in range(selected_chromo_data.shape[0]):
    L=selected_chromo_data.iloc[i]['loan size']
    # rL=selected_chromo_data.iloc[i]['interest']
    T = (1 - K) * D - L
    omega+=rT*T

  return omega

#beta
def cost_of_demand_deposit(rD,D):
    return rD * D

#sum of lambda
def sum_of_loss(selected_chromo_data):
  sum_of_lamda=0
  for i in range(selected_chromo_data.shape[0]):
    sum_of_lamda+=selected_chromo_data.iloc[i]['loss']
  return sum_of_lamda


def calc_fitness(selected_chromo_data,K,D,rD,rT):
  V=calc_revenue(selected_chromo_data)
  omega=calc_omega(selected_chromo_data,K,D,rT)
  beta=cost_of_demand_deposit(rD,D)
  sum_of_lamda=sum_of_loss(selected_chromo_data)

  total=V+omega-beta-sum_of_lamda
  return total



#calculate the fitness of entire population
#each row in population_fitness represents a chromosome in that population
def fitness_vector_of_population(population,customers_data,K,D,rD,rT):
  population_fitness=np.zeros(population.shape[0])

  for i in range(population.shape[0]):
    selected_candidates=[]
    for gene in range(len(population[i])):
      if population[i,gene]:
        selected_candidates.append(gene)
    
    selected_chromo_data=customers_data.iloc[selected_candidates,:]
    chromo_fitness=calc_fitness(selected_chromo_data,K,D,rD,rT)
    # print(chromo_fitness)

    population_fitness[i]=chromo_fitness
  return population_fitness



def roulette_wheel(pop_size,population,pop_fitness):
    cumulative=[]
    Parents=[]
    total_fitness=pop_fitness.sum()
    cumulative.append(pop_fitness[0]/total_fitness)
    for i in range(1,pop_size-1):
      cumulative.append(cumulative[i-1]+pop_fitness[i]/total_fitness)
    cumulative.append(1)
    for i in range(pop_size):
      # rand=np.random.uniform(0,1)
      rand=random.random()
      for j in range(pop_size):
        if rand < cumulative[j]:
          Parents.append(population[j])
          break
    # print(cumulative)
    return np.array(Parents)


#single pt crossover
def sp_cross(p1,p2,p_cross):
  # rand=np.random.uniform(0,1)
  rand=random.random()
  if rand<p_cross:
    c_point=np.random.randint(0,len(p1))
    f1=p1[:c_point]
    s1=p1[c_point:]
    f2=p2[:c_point]
    s2=p2[c_point:]
    child1=np.concatenate((f1,s2))
    child2=np.concatenate((f2,s1))
    return child1,child2
  else:
    return p1,p2

def crossover(number_of_customers,population_size,parents,p_cross):
  for i in range(population_size//2):
    parents[i],parents[population_size-i-1]=sp_cross(parents[i],parents[population_size-i-1],p_cross)
  return parents

def mutation2(number_of_customers,population_size,parents,K,D,p_mut):
  for i in range(population_size):
    for j in range(number_of_customers):
      # rand=np.random.uniform(0,1)
      rand=random.random()
      if rand < p_mut:
        parents[i][j]=(parents[i][j]==False)
  return parents


def modify_population(parents,init_best,customers_data,K,D):
  for i in range(len(parents)):
    chrom_index=chromosome_to_index(parents[i])
    if not check_limit(customers_data.iloc[chrom_index,:],K,D):
      temp=generate_initial_population(1,customers_data,len(parents),K,D)
      parents[i]=temp[0]
      # print("Not In limit")
      # parents[i]=init_best

  return parents


def chromosome_to_index(selected_chromo):
  df_index=[]
  for i in range(selected_chromo.shape[0]):
    if selected_chromo[i]:
      df_index.append(i)
  return df_index


In [ ]:
def Genetic_Algo(population,population_size,customers_data,number_of_customers,K,D,rD,rT,p_mut,p_cross,GA_iter,init_pop):
  g_best=0
  # g_sol=np.zeros(number_of_customers,dtype=bool)
  init_fitness=fitness_vector_of_population(population,customers_data,K,D,rD,rT)
  # print(init_fitness.max())
  init_best=population[init_fitness.argmax()]
  for i in range(GA_iter):
    pop_fitness=fitness_vector_of_population(population,customers_data,K,D,rD,rT)
    print(pop_fitness.max())
    g_best=max(pop_fitness.max(),g_best)
    g_sol=population[pop_fitness.argmax()]
    population=roulette_wheel(population_size,population,pop_fitness)
    population=crossover(number_of_customers,population_size,population,p_cross)
    population=mutation2(number_of_customers,population_size,population,K,D,p_mut)
    population=modify_population(population,init_best,customers_data,K,D)

    # population=modify_population(population,g_sol,customers_data,K,D)

  pop_fitness=fitness_vector_of_population(population,customers_data,K,D,rD,rT)
  print(pop_fitness.max())
  g_best=max(pop_fitness.max(),g_best)
  g_sol=population[pop_fitness.argmax()]

  print("GA BEST:",g_best)

  return g_best,g_sol,population



def SA(number_of_customers,population_size,number_of_iterations,K,D,rD,rT,p_cross,p_mut,initial_temp,c_factor,sa_iterations):
  Temp=initial_temp
  g_best=0
  g_best_sol=[]
  e=2.17
  init_pop=generate_initial_population(population_size,customers_data,number_of_customers,K,D)
  current_population=init_pop
  while Temp > 35:
    for i in range(sa_iterations):
        cur_best,cur_best_sol,ga_population=Genetic_Algo(current_population,population_size,customers_data,number_of_customers,K,D,rD,rT,p_mut,p_cross,GA_iter,init_pop)
        # rand_num = np.random.uniform(0, 1)
        rand=random.random()
        if cur_best>g_best:
          g_best=cur_best
          g_best_sol=cur_best_sol
          current_population=ga_population
          
        elif rand < np.power(e,((g_best-cur_best)/Temp)):
          current_population=ga_population
    Temp=Temp*c_factor               
    print("Current temp:",Temp)
  return g_best,g_best_sol

#Roulette Wheel Selection

In [ ]:
K=0.15
D=60
rD=0.009
rT=0.01
population_size=10
number_of_customers=10
number_of_iterations=10
p_cross=0.8
p_mut=0.006
GA_iter=10
initial_temp=500
c_factor=0.9
sa_iterations=5


# gb,gs,final_pop=Genetic_Algo(init_pop,population_size,customers_data,number_of_customers,K,D,rD,rT,p_mut,p_cross,GA_iter,init_pop)
# print("Best",gb)

gb,gs=SA(number_of_customers,population_size,GA_iter,K,D,rD,rT,p_cross,p_mut,initial_temp,c_factor,sa_iterations)

print("\n##############################################################################################################\n")
print("Final Result")
print("Best SOlution SA: ",gs)
print("Fmax: ",gb)



2.713
2.7175999999999996
2.7175999999999996
2.7175999999999996
2.713
2.7175999999999996
2.7175999999999996
2.713
2.7175999999999996
2.6124
2.6124
GA BEST: 2.7175999999999996
2.6124
2.6124
2.713
2.713
2.6828
2.6151999999999997
2.5154000000000005
2.5620000000000003
2.5154000000000005
2.5154000000000005
2.5154000000000005
GA BEST: 2.713
2.5154000000000005
2.5154000000000005
2.5154000000000005
2.5154000000000005
2.5154000000000005
2.5154000000000005
2.5154000000000005
2.5154000000000005
2.5154000000000005
3.1854
3.1854
GA BEST: 3.1854
3.1854
3.1854
3.1854
3.1854
3.1854
3.1854
3.1854
3.1854
3.1854
3.1854
2.6278
GA BEST: 3.1854
2.6278
2.6278
3.1854
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
GA BEST: 3.1854
Current temp: 450.0
2.6278
2.6278
2.6278
2.6278
2.6278
3.1854
2.6278
2.6278
2.6278
2.6278
2.6278
GA BEST: 3.1854
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
2.6278
GA BEST: 2.6278
2.6278
2.6278
3.1636
3.1636
3.1636
3.1636
2.6492000000000004
2.64920000

In [ ]:
init_pop=generate_initial_population(population_size,customers_data,number_of_customers,K,D)
# init_fitness=fitness_vector_of_population(init_pop,customers_data,K,D,rD,rT)
# print(init_pop[0])
gb,gs,final_pop=Genetic_Algo(init_pop,population_size,customers_data,number_of_customers,K,D,rD,rT,p_mut,p_cross,10,init_pop)
print("Best",gb)

2.6618
2.7872
2.7872
2.7872
2.6677999999999997
2.6677999999999997
2.7313999999999994
2.7313999999999994
2.6677999999999997
2.6677999999999997
2.6677999999999997
GA BEST: 2.7872
Best 2.7872


In [ ]:
init_pop=generate_initial_population(population_size,customers_data,number_of_customers,K,D)
gb,gs,final_pop=Genetic_Algo(init_pop,population_size,customers_data,number_of_customers,K,D,rD,rT,p_mut,p_cross,30,init_pop)
print("Best",gb)

2.1050000000000004
2.6626000000000003
2.6626000000000003
2.6626000000000003
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6324000000000005
2.6677999999999997
2.6677999999999997
3.1854
3.1854
3.1854
3.1854
3.1854
GA BEST: 3.1854
Best 3.1854
